In [1]:
import pandas as pd
import numpy as np
import random
import scipy
import scipy as sp
import scipy.stats as sts
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(42)

 <a id='eda'></a>
### Первичный анализ данных

In [2]:
df = pd.read_csv('ab_datas.csv')

df.head()

,user_id,timestamp,group,landing_page,converted,revenue
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,100
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,250


In [3]:
df[df['revenue']==100].describe()

,user_id,converted,revenue
count,85615.000000,85615.000000,85615.0
mean,787998.753583,0.119138,100.0
std,91177.345717,0.323953,0.0
min,630000.000000,0.000000,100.0
25%,709194.500000,0.000000,100.0
50%,788136.000000,0.000000,100.0
75%,866785.500000,0.000000,100.0
max,945999.000000,1.000000,100.0


In [4]:
df[df['revenue']==250].describe()

,user_id,converted,revenue
count,85863.000000,85863.000000,85863.0
mean,787994.834108,0.120168,250.0
std,91257.312018,0.325160,0.0
min,630003.000000,0.000000,250.0
25%,708856.500000,0.000000,250.0
50%,788027.000000,0.000000,250.0
75%,867038.000000,0.000000,250.0
max,945993.000000,1.000000,250.0


In [5]:
df['revenue'].value_counts()

0      123000
250     85863
100     85615
Name: revenue, dtype: int64

In [7]:
#объем тестовой группы
n_rows = df.shape[0]
n_rows_treat = len(df[df['group'] == 'treatment'])

#объем контрольной группы
n_rows_contr = n_rows-n_rows_treat

print("Соотношение размеров тестовой и контрольной групп: {}".format(n_rows_treat/n_rows_contr))

print("Вероятность, что новый пользователь будет направлен на версию со старым дизайном: {}".format(n_rows_treat/n_rows))
print("Вероятность, что новый пользователь будет направлен на версию с новым дизайном: {}".format(n_rows_contr/n_rows))


Соотношение размеров тестовой и контрольной групп: 1.0005027105609978
Вероятность, что новый пользователь будет направлен на версию со старым дизайном: 0.5001256460584492
Вероятность, что новый пользователь будет направлен на версию с новым дизайном: 0.49987435394155083


<a id='si4'></a>
#### Проведение вычислений. $Z$-критерий.

In [9]:
#Z-статистика
import statsmodels.api as sm
import statsmodels

convert_contr = sum(df.query("revenue == 100 ")['converted'])
convert_treat = sum(df.query("revenue == 250")['converted'])

z_score, p_value = sm.stats.proportions_ztest([convert_treat, convert_contr], [n_rows_treat, n_rows_contr], 
                                              alternative='larger')

print("Z-статистика={},  p_value={}".format(z_score, p_value))


Z-статистика=0.8167587147214324,  p_value=0.20703316498053792


#### Проведение вычислений. $T$-test.

In [10]:
t_score, p_value, _ = statsmodels.stats.weightstats.ttest_ind(df.query("group == 'treatment'")['converted'],
                                                              df.query("group == 'control'")['converted'],
                                                              alternative="larger",
                                                              usevar='unequal')

In [11]:
print("T-статистика={},  p_value={}".format(t_score, p_value))

T-статистика=-1.2369191070650987,  p_value=0.8919409489056582


#### Проведение вычислений. $Mann–Whitney$-test.

In [12]:
w_score, p_value = scipy.stats.mannwhitneyu(df.query("group == 'treatment'")['converted'], 
                         df.query("group == 'control'")['converted'],
                         alternative="greater")

In [13]:
print("W-статистика={},  p_value={}".format(w_score, p_value))

W-статистика=10823622516.0,  p_value=0.8919415509213815
